In [1]:
from unet_project.vessels_functions import load_model, predict_images
from unet_project.image_utils import ImageUtils
import os  
import random
import cv2
import numpy as np
from matplotlib import pyplot as plt
from segmentation_models.backbones import get_preprocessing
from keras import backend as K

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [2]:
#!pip3 install pymia

In [3]:
val_frame_path = '/mnt/data/aja/diplomka/custom_train/val_imgs/val'
val_mask_path = '/mnt/data/aja/diplomka/custom_train/val_masks/val'
img_height = 512
img_width = 512

image_ut = ImageUtils(val_frame_path, val_mask_path, img_height, img_width, architecture='unet')

In [4]:
from scipy.signal import medfilt2d
from skimage import data, color
from skimage import measure

In [5]:
linknet_resnet_150 =  load_model('model_linknet_vessels_pretrainedRESNET18_150noeqa')

unet_resnet18_150 =  load_model('model_unet_vessels_pretrainedRESNET18_150noeqa')

Loaded model from disk
Loaded model from disk


In [6]:
imgs_linknet_resnet_150, masks_linknet_resnet_150, predicted_imgs_linknet_resnet_150 = predict_images(backbone='resnet18', 
                                                                                      val_frame_path=val_frame_path,
                                                                                      val_mask_path=val_mask_path,
                                                                                      img_height=img_height,
                                                                                      img_width=img_width,
                                                                                      model=linknet_resnet_150,
                                                                                      preprocess=True)
imgs_unet_resnet18_150, masks_unet_resnet18_150, predicted_imgs_unet_resnet18_150 = predict_images(backbone='resnet18', 
                                                                                      val_frame_path=val_frame_path,
                                                                                      val_mask_path=val_mask_path,
                                                                                      img_height=img_height,
                                                                                      img_width=img_width,
                                                                                      model=unet_resnet18_150,
                                                                                      preprocess=True)

In [7]:
def tresholding(imgs, treshold):
    for m in imgs:
        m[m>treshold]=1
        m[m<=treshold]=0
    return imgs

def sensitivity(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = np.sum(np.round(np.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = np.sum(np.round(np.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def dice_coef(y_true, y_pred, smooth=1):
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    return np.mean((2. * intersection + smooth) / (np.sum(np.square(y_true),-1) + np.sum(np.square(y_pred),-1) + smooth))

## Linknet

In [8]:
masks_linknet_resnet_150 = tresholding(masks_linknet_resnet_150, 0.02)
predicted_imgs_linknet_resnet_150 = tresholding(predicted_imgs_linknet_resnet_150, 0.02)
predicted_imgs_linknet_resnet_150 = np.squeeze(np.squeeze(np.array(predicted_imgs_linknet_resnet_150)))

In [9]:
Sens = sensitivity(np.array(masks_linknet_resnet_150), np.array(predicted_imgs_linknet_resnet_150))
Spec = specificity(np.array(masks_linknet_resnet_150), np.array(predicted_imgs_linknet_resnet_150))
Dice = dice_coef(np.array(masks_linknet_resnet_150), np.array(predicted_imgs_linknet_resnet_150))

In [10]:
print('LinkNet sensitivity: {}'. format(Sens))
print('LinkNet specifity: {}'. format(Spec))
print('LinkNet dice coefficient: {}'. format(Dice))

LinkNet sensitivity: 0.9249141592567922
LinkNet specifity: 0.8613708365789432
LinkNet dice coefficient: 0.6820310672711453


## Unet

In [11]:
masks_unet_resnet18_150 = tresholding(masks_unet_resnet18_150, 0.02)
predicted_imgs_unet_resnet18_150 = tresholding(predicted_imgs_unet_resnet18_150, 0.02)
predicted_imgs_unet_resnet18_150 = np.squeeze(np.squeeze(np.array(predicted_imgs_unet_resnet18_150)))

In [12]:
Sens = sensitivity(np.array(masks_unet_resnet18_150), np.array(predicted_imgs_unet_resnet18_150))
Spec = specificity(np.array(masks_unet_resnet18_150), np.array(predicted_imgs_unet_resnet18_150))
Dice = dice_coef(np.array(masks_unet_resnet18_150), np.array(predicted_imgs_unet_resnet18_150))

In [13]:
print('Unet sensitivity: {}'. format(Sens))
print('Unet specifity: {}'. format(Spec))
print('Unet dice coefficient: {}'. format(Dice))

Unet sensitivity: 0.9090217422031653
Unet specifity: 0.8858327758713873
Unet dice coefficient: 0.7157163632682361
